In [2]:
import urllib.request as urllib2
from bs4 import BeautifulSoup
import pandas as pd
import re
from unidecode import unidecode
import os

In [11]:
# get the link to the metrolyrics page of your artist

page_link_format = 'http://www.metrolyrics.com/eminem-alpage-{}.html'

songs = []
for i in range(1, 10):
    page_link = page_link_format.format(str(i))
    print(page_link)
    page = urllib2.urlopen(page_link)
    soup = BeautifulSoup(page, 'html.parser')
    songs.extend(soup.find_all('a', attrs={'class': 'title hasvidtable'}))
    
# extract all the song titles you can find
link_lyric_list = []
for song in songs:
    link_lyric = re.findall(r"(http.*html)", (str(song)))[0]
    link_lyric_list.append(link_lyric)

df_lyrics = pd.DataFrame(link_lyric_list, columns=['link'])

http://www.metrolyrics.com/eminem-alpage-1.html
http://www.metrolyrics.com/eminem-alpage-2.html
http://www.metrolyrics.com/eminem-alpage-3.html
http://www.metrolyrics.com/eminem-alpage-4.html
http://www.metrolyrics.com/eminem-alpage-5.html
http://www.metrolyrics.com/eminem-alpage-6.html
http://www.metrolyrics.com/eminem-alpage-7.html
http://www.metrolyrics.com/eminem-alpage-8.html
http://www.metrolyrics.com/eminem-alpage-9.html


In [12]:
# itterate through df and get all the text from the link

for index, row in df_lyrics.iterrows():
    page = urllib2.urlopen(row['link'])
    soup = BeautifulSoup(page, 'html.parser')
    verses = soup.find_all('p', attrs={'class': 'verse'})
    title_element = soup.find_all('title')

    title = re.findall(r"(?<=<title>).*?(?=\sLyrics)" , str(title_element))[0]

    
    lyrics = ''
    for verse in verses:
        text = verse.text.strip()
        text = re.sub(r"\[.*\]\n", "", unidecode(text))
        if lyrics == '':
            lyrics = lyrics + text.replace('\n', ' ')
        else:
            lyrics = lyrics + ' ' + text.replace('\n', ' ')

    df_lyrics.at[index, 'lyrics'] = lyrics
    df_lyrics.at[index, 'title'] = title

    print('writing: ', title )

writing:  Eminem - '97 Bonnie And Clyde
writing:  Eminem - 1 Shot 2 Shot
writing:  Eminem - 14) Amityville
writing:  Eminem - 25 To Life
writing:  Eminem - 25 To Life (Remix)
writing:  Eminem - 3 A.m.
writing:  Eminem - 313
writing:  Eminem - 50 Ways
writing:  Eminem - 8 Mile
writing:  Eminem - 8 Mile Freestyle Pt.III Vs
writing:  Eminem - 8 Mile Road
writing:  Eminem - A Drop In The Ocean
writing:  Eminem - A Kiss
writing:  Eminem - Above The Law
writing:  Eminem - Airplanes Part 2
writing:  Eminem - All She Wrote
writing:  Eminem - All She Wrote
writing:  Eminem - Almost Famous
writing:  Eminem - American Psycho
writing:  Eminem - Amityville
writing:  Eminem - Any Man
writing:  Eminem - Arose
writing:  Eminem - As The World Turns
writing:  Eminem - Ass Like That
writing:  Eminem - Asshole
writing:  Eminem - Baby
writing:  Eminem - Back Stabber
writing:  Eminem - Bad Guy
writing:  Eminem - Bad Guys Always Die
writing:  Eminem - Bad Husband
writing:  Eminem - Bad Influence
writing:  Em

writing:  Eminem - My Name Is (Real Dirty)
writing:  Eminem - My Words Are Weapons
writing:  Eminem - Nail In The Coffin
writing:  Eminem - Need A Doctor
writing:  Eminem - Need Me
writing:  Eminem - Never 2 Far
writing:  Eminem - Never Enough
writing:  Eminem - Nice Guy
writing:  Eminem - No Apologies
writing:  Eminem - No Love
writing:  Eminem - No One's Iller
writing:  Eminem - No One's Iller Than Me
writing:  Eminem - No One's Iller Then Me
writing:  Rihanna - Love The Way You Lie Part II
writing:  Akon - Smack That
writing:  Lil Wayne - Drop The World
writing:  Dr. Dre - Forgot About Dre -
writing:  Rihanna - Love The Way You Lie
writing:  P!nk - Revenge
writing:  Dr. Dre - Forget About Dre
writing:  Dr. Dre - What's The Difference
writing:  Royce Da 5'9" - Caterpillar
writing:  Dr. Dre - I Need A Doctor
writing:  50 Cent - My Life
writing:  Eminem - Normal
writing:  Eminem - Not Affraid
writing:  Eminem - Not Afraid
writing:  Eminem - Not Alike
writing:  Eminem - Nowhere Fast
wri

In [13]:
print(df_lyrics.head())
print(df_lyrics.shape)

                                                link  \
0  http://www.metrolyrics.com/97-bonnie-and-clyde...   
1  http://www.metrolyrics.com/1-shot-2-shot-lyric...   
2  http://www.metrolyrics.com/14-amityville-lyric...   
3  http://www.metrolyrics.com/25-to-life-lyrics-e...   
4  http://www.metrolyrics.com/25-to-life-remix-ly...   

                                              lyrics  \
0  Baby, your dada loves you And I'ma always be h...   
1                                                      
2  Kill Dah-dum, dah-dum, dum Dah-dum, dah-dum, d...   
3  Too late for the other side Caught in a chase ...   
4  Too late for the other side Caught in a chase ...   

                           title  
0  Eminem - '97 Bonnie And Clyde  
1         Eminem - 1 Shot 2 Shot  
2        Eminem - 14) Amityville  
3            Eminem - 25 To Life  
4    Eminem - 25 To Life (Remix)  
(432, 3)


In [14]:
# drop columns with songs not only by Eminem
mask = df_lyrics['title'].str.contains('Eminem', regex=False)
df_lyrics = df_lyrics[mask]
print(df_lyrics.shape)

(332, 3)


In [15]:
# check if folder exists
directory = "files"
if not os.path.exists(directory):
    os.makedirs(directory)
    

filename = directory + "/eminem_lyrics.csv"

print('writing to .csv')
df_lyrics.to_csv(filename, sep=';', encoding='utf-8')

writing to .csv
